In [2]:
import scipy
import scipy.signal
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

import xlrd

In [36]:
import scipy
import scipy.signal
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import xlrd


def matrix_from_csv_file(file_path):
    
    csv_data = np.genfromtxt(file_path, delimiter = ',')
    full_matrix = csv_data[2:] 
    return full_matrix


def get_slice (full_matrix,  t, frame):
    tot_len = full_matrix[:,0].size 
    
    restart = full_matrix[0, 0] + t
    index_0 = int(restart)
    index_1 = int(restart + frame)
    print('index_0', index_0)
    print('index_1', index_1)
    duration = frame#full_matrix[index_1, 0] - full_matrix[index_0, 0]
    #print('Duration is', duration )
    #print('FULL MATRIX ', full_matrix[index_0:index_1, :])
    return full_matrix [index_0:index_1, :], duration


def notch_filter(x, samplerate, plot=False):
    x = x - np.mean(x)

    high_cutoff_notch = 49 / (samplerate / 2)
    low_cutoff_notch = 51 / (samplerate / 2)

    # Band Stop Filter (BSF) or Band Reject Filter
    [b, a] = signal.butter(4, [high_cutoff_notch, low_cutoff_notch], btype='stop')

    x_filt = signal.filtfilt(b, a, x.T)

    if plot:
        t = np.arange(0, len(x) / samplerate, 1 / samplerate)
        plt.plot(t, x)
        plt.plot(t, x_filt.T, 'k')
        plt.autoscale(tight=True)
        plt.xlabel('Time')
        plt.ylabel('Amplitude (mV)')
        plt.show()

    return x_filt


def bp_filter(x, low_f, high_f, samplerate, plot=False):
    # x = x - np.mean(x)

    low_cutoff_bp = low_f / (samplerate / 2)
    high_cutoff_bp = high_f / (samplerate / 2)

    [b, a] = signal.butter(5, [low_cutoff_bp, high_cutoff_bp], btype='bandpass')

    x_filt = signal.filtfilt(b, a, x)
    #print(len(x))

    if plot:
        t = np.arange(0, len(x.transpose()) / samplerate, 1 / samplerate)
        plt.plot(t, x.transpose())
        plt.plot(t, x_filt.transpose(), 'k')
        plt.autoscale(tight=True)
        plt.xlabel('Time')
        plt.ylabel('Amplitude (mV)')
        plt.show()

    return x_filt


def feature_mean(matrix):
  
    ret = np.mean(matrix, axis = 0).flatten()
    #print('MEAN RET VALUE',ret )
    names = ['mean_' + str(i) for i in range(matrix.shape[1])]
    return ret, names


def feature_mean_d(h1, h2):

    ret = (feature_mean(h2)[0] - feature_mean(h1)[0]).flatten()


    # Fixed naming [fcampelo]
    names = ['mean_d_h2h1_' + str(i) for i in range(h1.shape[1])]
    return ret, names


def feature_mean_q(q1, q2, q3, q4):

    v1 = feature_mean(q1)[0]
    v2 = feature_mean(q2)[0]
    v3 = feature_mean(q3)[0]
    v4 = feature_mean(q4)[0]
    ret = np.hstack([v1, v2, v3, v4, 
                     v1 - v2, v1 - v3, v1 - v4, 
                     v2 - v3, v2 - v4, v3 - v4]).flatten()


    # Fixed naming [fcampelo]
    names = []
    for i in range(4): # for all quarter-windows
        names.extend(['mean_q' + str(i + 1) + "_" + str(j) for j in range(len(v1))])

    for i in range(3): # for quarter-windows 1-3
        for j in range((i + 1), 4): # and quarter-windows (i+1)-4
            names.extend(['mean_d_q' + str(i + 1) + 'q' + str(j + 1) + "_" + str(k) for k in range(len(v1))])

    return ret, names


def feature_stddev(matrix):

    # fix ddof for finite sampling correction (N-1 instead of N in denominator)
    ret = np.std(matrix, axis = 0, ddof = 1).flatten()
    names = ['std_' + str(i) for i in range(matrix.shape[1])]

    return ret, names


def feature_stddev_d(h1, h2):
 
    ret = (feature_stddev(h2)[0] - feature_stddev(h1)[0]).flatten()

    # Fixed naming [fcampelo]
    names = ['std_d_h2h1_' + str(i) for i in range(h1.shape[1])]

    return ret, names


def feature_moments(matrix):
 
    skw = scipy.stats.skew(matrix, axis = 0, bias = False)
    krt = scipy.stats.kurtosis(matrix, axis = 0, bias = False)
    ret  = np.append(skw, krt)
    #print('MOMENTS SHAPE' , ret.shape)
    names = ['skew_' + str(i) for i in range(matrix.shape[1])]
    names.extend(['kurt_' + str(i) for i in range(matrix.shape[1])])
    return ret, names


def feature_max(matrix):
  
    ret = np.max(matrix, axis = 0).flatten()
    names = ['max_' + str(i) for i in range(matrix.shape[1])]
    return ret, names


def feature_max_d(h1, h2):
   
    ret = (feature_max(h2)[0] - feature_max(h1)[0]).flatten()

    # Fixed naming [fcampelo]
    names = ['max_d_h2h1_' + str(i) for i in range(h1.shape[1])]
    return ret, names


def feature_max_q(q1, q2, q3, q4):
 
    v1 = feature_max(q1)[0]
    v2 = feature_max(q2)[0]
    v3 = feature_max(q3)[0]
    v4 = feature_max(q4)[0]
    ret = np.hstack([v1, v2, v3, v4, 
                     v1 - v2, v1 - v3, v1 - v4, 
                     v2 - v3, v2 - v4, v3 - v4]).flatten()

    # Fixed naming [fcampelo]
    names = []
    for i in range(4): # for all quarter-windows
        names.extend(['max_q' + str(i + 1) + "_" + str(j) for j in range(len(v1))])

    for i in range(3): # for quarter-windows 1-3
        for j in range((i + 1), 4): # and quarter-windows (i+1)-4
            names.extend(['max_d_q' + str(i + 1) + 'q' + str(j + 1) + "_" + str(k) for k in range(len(v1))])

    return ret, names


def feature_min(matrix):
  
    ret = np.min(matrix, axis = 0).flatten()
    names = ['min_' + str(i) for i in range(matrix.shape[1])]
    return ret, names



def feature_min_d(h1, h2):
    """
    Computes the change in min values (backward difference) of all signals 
    between the first and second half-windows, min(h2) - min(h1)
   
    """

    ret = (feature_min(h2)[0] - feature_min(h1)[0]).flatten()

    # Fixed naming [fcampelo]
    names = ['min_d_h2h1_' + str(i) for i in range(h1.shape[1])]
    return ret, names


def feature_min_q(q1, q2, q3, q4):
    """
    Computes the min values of each signal for each quarter-window, plus the 
    paired differences of min values of each signal for the quarter-windows, 
    i.e., feature_min(q1), feature_min(q2), feature_min(q3), feature_min(q4),
    (feature_min(q1) - feature_min(q2)), (feature_min(q1) - feature_min(q3)),
    ...

    """
    v1 = feature_min(q1)[0]
    v2 = feature_min(q2)[0]
    v3 = feature_min(q3)[0]
    v4 = feature_min(q4)[0]
    ret = np.hstack([v1, v2, v3, v4, 
                     v1 - v2, v1 - v3, v1 - v4, 
                     v2 - v3, v2 - v4, v3 - v4]).flatten()


    # Fixed naming [fcampelo]
    names = []
    for i in range(4): # for all quarter-windows
        names.extend(['min_q' + str(i + 1) + "_" + str(j) for j in range(len(v1))])

    for i in range(3): # for quarter-windows 1-3
        for j in range((i + 1), 4): # and quarter-windows (i+1)-4
            names.extend(['min_d_q' + str(i + 1) + 'q' + str(j + 1) + "_" + str(k) for k in range(len(v1))])

    return ret, names


def feature_covariance_matrix(matrix):
    """
    Computes the elements of the covariance matrix of the signals. Since the 
    covariance matrix is symmetric, only the lower triangular elements 
    (including the main diagonal elements, i.e., the variances of eash signal) 
    are returned. 

 
    """

    covM = np.cov(matrix.T)
    indx = np.triu_indices(covM.shape[0])
    ret  = covM[indx]

    names = []
    for i in np.arange(0, covM.shape[1]):
        for j in np.arange(i, covM.shape[1]):
            names.extend(['covM_' + str(i) + '_' + str(j)])
    #print('COVM VALUE ',ret )
    return ret, names, covM


def feature_eigenvalues(covM):
    """
    Computes the eigenvalues of the covariance matrix passed as the function 
    argument.

    Parameters:
        covM (numpy.ndarray): 2D [nsignals x nsignals] covariance matrix of the 
        signals in a time window

  
    """

    ret   = np.linalg.eigvals(covM).flatten()
    names = ['eigenval_' + str(i) for i in range(covM.shape[0])]
    #print('EIGRN VALUE',ret )
    return ret, names


def feature_logcov(covM):
    """
    Computes the matrix logarithm of the covariance matrix of the signals. 
      
    """
    log_cov = scipy.linalg.logm(covM)
    indx = np.triu_indices(log_cov.shape[0])
    ret  = np.abs(log_cov[indx])

    names = []
    for i in np.arange(0, log_cov.shape[1]):
        for j in np.arange(i, log_cov.shape[1]):
            names.extend(['logcovM_' + str(i) + '_' + str(j)])

    return ret, names, log_cov



def feature_fft(matrix, period = 1., mains_f = 50., 
                filter_mains = True, filter_DC = True,
                normalise_signals = True,
                ntop = 10, get_power_spectrum = True):
   
    # Signal properties
    N   = matrix.shape[0] # number of samples
    T = period / N        # Sampling period

    # Scale all signals to interval [-1, 1] (if requested)
    if normalise_signals:
        matrix = -1 + 2 * (matrix - np.min(matrix)) / (np.max(matrix) - np.min(matrix))

    # Compute the (absolute values of the) FFT
    # Extract only the first half of each FFT vector, since all the information
    # is contained there (by construction the FFT returns a symmetric vector).
    fft_values = np.abs(scipy.fft.fft(matrix, axis = 0))[0:N//2] * 2 / N

    # Compute the corresponding frequencies of the FFT components
    freqs = np.linspace(0.0, 1.0 / (2.0 * T), N//2)

    # Remove DC component (if requested)
    if filter_DC:
        fft_values = fft_values[1:]
        freqs = freqs[1:]

    # Remove mains frequency component(s) (if requested)
    if filter_mains:
        indx = np.where(np.abs(freqs - mains_f) <= 1)
        fft_values = np.delete(fft_values, indx, axis = 0)
        freqs = np.delete(freqs, indx)

    # Extract top N frequencies for each signal
    indx = np.argsort(fft_values, axis = 0)[::-1]
    indx = indx[:ntop]

    ret = freqs[indx].flatten(order = 'F')

    # Make feature names
    names = []
    for i in np.arange(fft_values.shape[1]):
        names.extend(['topFreq_' + str(j) + "_" + str(i) for j in np.arange(1,11)])

    if (get_power_spectrum):
        ret = np.hstack([ret, fft_values.flatten(order = 'F')])

        for i in np.arange(fft_values.shape[1]):
            names.extend(['freq_' + "{:03d}".format(int(j)) + "_" + str(i) for j in 10 * np.round(freqs, 1)])
            #print ('Names', names)
    return ret, names


def calc_feature_vector(matrix, state):
    """
    Calculates all previously defined features and concatenates everything into 
    a single feature vector.


    """

    # Extract the half- and quarter-windows
    h1, h2 = np.split(matrix, [ int(matrix.shape[0] / 2) ])
    q1, q2, q3, q4 = np.split(matrix, 
                              [int(0.25 * matrix.shape[0]), 
                               int(0.50 * matrix.shape[0]), 
                               int(0.75 * matrix.shape[0])])

    var_names = []	

    x, v = feature_mean(matrix)
    var_names += v
    var_values = x
    
    
    x, v = feature_mean_d(h1, h2)
    var_names += v
    var_values = np.hstack([var_values, x])
    
    '''
    x, v = feature_mean_q(q1, q2, q3, q4) # per channel 10 features
    var_names += v
    #print('this Feature shape', var_values.shape)
    var_values = np.hstack([var_values, x])
    '''
    
    x, v = feature_stddev(matrix)
    var_names += v
    var_values = np.hstack([var_values, x])
    

    x, v = feature_stddev_d(h1, h2)
    var_names += v
    var_values = np.hstack([var_values, x])
    
    x, v = feature_moments(matrix)  # per channel 2 features
    var_names += v
    var_values = np.hstack([var_values, x])
    
    x, v = feature_max(matrix)
    var_names += v
    var_values = np.hstack([var_values, x])
    
    x, v = feature_max_d(h1, h2)
    var_names += v
    var_values = np.hstack([var_values, x])
    
    '''
    x, v = feature_max_q(q1, q2, q3, q4) # per channel 10 features
    var_names += v
    var_values = np.hstack([var_values, x])
    '''

    x, v = feature_min(matrix)
    var_names += v
    var_values = np.hstack([var_values, x])

    x, v = feature_min_d(h1, h2)
    var_names += v
    var_values = np.hstack([var_values, x])
    ''''
    x, v = feature_min_q(q1, q2, q3, q4)
    var_names += v
    var_values = np.hstack([var_values, x])
   
    x, v, covM = feature_covariance_matrix(matrix) # 10 Fea per channel
    var_names += v
    var_values = np.hstack([var_values, x])

    x, v = feature_eigenvalues(covM) # 4 fea per channel
    var_names += v
    var_values = np.hstack([var_values, x])
    
    
    x, v, log_cov = feature_logcov(covM)
    var_names += v
    var_values = np.hstack([var_values, x])
    
   
    x, v = feature_fft(matrix)
    var_names += v
    var_values = np.hstack([var_values, x]) 
    #print('Feature  shape', var_values.shape)
    '''
    if state != None:
        var_values = np.hstack([var_values, np.array([state])])
        var_names += ['Label']
    #print('var_values', var_values)
    #print('var_names', var_names)
    #print('all val values shape', var_values.shape)
    return var_values, var_names



"""
Returns a number of feature vectors from a labeled CSV file, and a CSV header 

"""
def generate_feature_vectors_from_samples(file_path, nsamples, period, 
                                          state = None, 
                                          remove_redundant = True,
                                          cols_to_ignore = None):
    """
    Reads data from CSV file in "file_path" and extracts statistical features 
    for each time window of width "period". 
  
    """	
    # Read the matrix from file
    matrix = matrix_from_csv_file(file_path)
    samplerate = 2000
    #samplerate = 2000
    low_f = 1
    high_f = 500
    
    t = 0.

    # No previous vector is available at the start
    previous_vector = None

    # Initialise empty return object
    ret = None
    
    s,dur = get_slice (matrix,  t, nsamples)
    #print('s  slice SHAPE first ', s.shape)
    # Until an exception is raised or a stop condition is met
    while True:
        # Get the next slice from the file (starting at time 't', with a 
        # duration of 'period'
        # If an exception is raised or the slice is not as long as we expected, 
        # return the current data available
        try:
            
            t=t+(nsamples/2)
            s,dur = get_slice (matrix, t,nsamples)
            
            #print('s slice shape in try ', s.shape)
            if cols_to_ignore is not None:
                s = np.delete(s, cols_to_ignore, axis = 1)
        except IndexError:
            break
        if len(s) == 0:
            #print('Length is zero')
            break
        if dur < 0.9 * period:
            #print(' Duration is less')
            break

        # Perform the resampling of the vector
                
        #sig = (s[:, 1:])
        # Perform the resampling of the vector

        sig1, rx = scipy.signal.resample(s[:, 1:], num = nsamples,  t = s[:, 0], axis =0)
        
        #print(' sig1 notch ip shape is', sig1.shape)
        # Notch Filter 
        sig2 = notch_filter(sig1, samplerate,  False)
        #print(' sig2 notch op shape is', sig2.shape)
        
        # Band Pass Filter (BPF)
        ry = bp_filter(sig2, 10, 49,  samplerate,  False)
        #print(' ry BP output shape is', ry.shape)                    
        
        #t += 0.5 * period
        # Compute the feature vector. We will be appending the features of the 
        # current time slice and those of the previous one.
        # If there was no previous vector we just set it and continue 
        # with the next vector.
        #
        #print( 'sig2 shape', sig2.shape)
        r, headers = calc_feature_vector( ry.transpose(), state)#ry.transpose()
           

        if previous_vector is not None:
            # If there is a previous vector, the script concatenates the two 
            # vectors and adds the result to the output matrix
            feature_vector = np.hstack([previous_vector, r])

            if ret is None:
                ret = feature_vector
            else:
                ret = np.vstack([ret, feature_vector])

        # Store the vector of the previous windowheaders 
        previous_vector = r
        if state is not None:
            # Remove the label (last column) of previous vector
            previous_vector = previous_vector[:-1] 
    #print('out of the loop')
    feat_names = ["lag1_" + s for s in headers[:-1]] + headers

    # Return
    return ret, feat_names



# ========================================================================
# ========================================================================
# ========================================================================



import os, sys
import numpy as np
# from EEG_feature_extraction import generate_feature_vectors_from_samples
#directory_path = './EEG_Emotion_data/'

import pandas as pd
import os
   

#directory_path='C:/Users/ajay/Documents/EEG_Emotion/EEG_Emotion_data'
#os.listdir(directory_path)


def gen_training_matrix(directory_path, output_file, cols_to_ignore):
    """
    Reads the csv files in directory_path and assembles the training matrix with 
    the features extracted using the functions from EEG_feature_extraction.

 
    """

    # Initialise return matrix
    FINAL_MATRIX = None

    for x in os.listdir(directory_path):
        print ('x is' , x)
        #s=0
        #while s < no_of_folders:
        sub_folder_path = directory_path  +   '/'
        print( 'Sub folder path is', sub_folder_path)
        for sub in os.listdir(sub_folder_path):
            #print ('s is' , sub)
            print ('Using file', sub)
            full_file_path = sub_folder_path  +   '/'   + sub

            try:
                name, state, _ = sub[:-4].split('-')

            except:
                print ('Wrong file name', sub)
                sys.exit(-1)
            if state.lower() == 'old':
                state = 1.
            elif state.lower() == 'young':
                state = 0.
            else:
                print ('Wrong file name', sub)
                sys.exit(-1)

            print('STATE IS', state)
            
            vectors, header = generate_feature_vectors_from_samples( file_path = full_file_path,
                                                                    nsamples = 600,
                                                                    period = 1.,
                                                                    state = state,
                                                                    remove_redundant = True,
                                                                    cols_to_ignore = cols_to_ignore)
            
            #vectors, header = generate_feature_vectors_from_samples( file_path = full_file_path, 
                                                                    #nsamples = 10, 
                                                                    #period = 1.,
                                                                    #state = state,
                                                                    #remove_redundant = True,
                                                                    #cols_to_ignore = cols_to_ignore)

            #print ('resulting vector shape for the file', vectors.shape)
            TEMP_MATRIX = vectors
            TEMP_MATRIX = np.vstack( [  TEMP_MATRIX, vectors ] )
            #print('TEMP_MATRIX', TEMP_MATRIX.shape)
            #print('TEMP_MATRIX', TEMP_MATRIX )#.shape)
            if FINAL_MATRIX is None:
                FINAL_MATRIX = vectors
                print ('FINAL_MATRIX SHAPE first', FINAL_MATRIX.shape)
            else:
                FINAL_MATRIX = np.vstack( [ FINAL_MATRIX, vectors ] )
                print ('FINAL_MATRIX SHAPE consecutive', FINAL_MATRIX.shape)
            print ('FINAL_MATRIX SHAPE final', FINAL_MATRIX.shape)

        # Shuffle rows
        #np.random.shuffle(FINAL_MATRIX)

        # Save to file
        np.savetxt(output_file, FINAL_MATRIX, delimiter = ',', header = ','.join(header), comments = '')

        return None


if __name__ == '__main__':
   

    #directory_path='C:/Users/ajay/Documents/EEG_Emotion/EEG_Emotion_data'
    #output_file='C:/Users/ajay/Documents/EEG_Emotion/out.csv'

    directory_path='C:/Users/jagda/OneDrive/Documents/ML_dataset/Train'
    output_file='C:/Users/jagda/OneDrive/Documents/ML_dataset/Train.csv'
    gen_training_matrix(directory_path, output_file, cols_to_ignore = None)#-1)
    #C:\Users\ajay\Documents\HPID40
    
    directory_path='C:/Users/jagda/OneDrive/Documents/ML_dataset/Test'
    output_file='C:/Users/jagda/OneDrive/Documents/ML_dataset/Test.csv'
    gen_training_matrix(directory_path, output_file, cols_to_ignore = None)#-1)


    
    

x is 1-old-all.csv
Sub folder path is C:/Users/jagda/OneDrive/Documents/ML_dataset/Train/
Using file 1-old-all.csv
STATE IS 1.0
index_0 -2
index_1 598
index_0 298
index_1 898
index_0 598
index_1 1198
index_0 898
index_1 1498
index_0 1198
index_1 1798
index_0 1498
index_1 2098
index_0 1798
index_1 2398
index_0 2098
index_1 2698
index_0 2398
index_1 2998
index_0 2698
index_1 3298
index_0 2998
index_1 3598
index_0 3298
index_1 3898
index_0 3598
index_1 4198
index_0 3898
index_1 4498
index_0 4198
index_1 4798
index_0 4498
index_1 5098
index_0 4798
index_1 5398
index_0 5098
index_1 5698
index_0 5398
index_1 5998
index_0 5698
index_1 6298
index_0 5998
index_1 6598
index_0 6298
index_1 6898
index_0 6598
index_1 7198
index_0 6898
index_1 7498
index_0 7198
index_1 7798
index_0 7498
index_1 8098
index_0 7798
index_1 8398
index_0 8098
index_1 8698
index_0 8398
index_1 8998
index_0 8698
index_1 9298
index_0 8998
index_1 9598
index_0 9298
index_1 9898
index_0 9598
index_1 10198
index_0 9898
index_1

In [2]:

import numpy as np
import pandas as pd
import os
import csv
import cv2
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score

In [3]:
#Train_set = pd.read_csv('C:/Users/ajay/Documents/EEG_Emotion/HPID40_Train_EEG_SIT_STAND.csv')
Train_set = pd.read_csv('C:/Users/jagda/OneDrive/Documents/ML_dataset/Train.csv')


#Test_set  = pd.read_csv('C:/Users/ajay/Documents/EEG_Emotion/HPID40_Test_EEG_SIT_STAND.csv')
Test_set  = pd.read_csv('C:/Users/jagda/OneDrive/Documents/ML_dataset/Test.csv')

In [39]:
Train_set 

,lag1_mean_0,lag1_mean_1,lag1_mean_2,lag1_mean_d_h2h1_0,lag1_mean_d_h2h1_1,lag1_mean_d_h2h1_2,lag1_std_0,lag1_std_1,lag1_std_2,lag1_std_d_h2h1_0,...,max_d_h2h1_0,max_d_h2h1_1,max_d_h2h1_2,min_0,min_1,min_2,min_d_h2h1_0,min_d_h2h1_1,min_d_h2h1_2,Label
0,-0.078129,0.016685,-0.046109,0.008124,0.094784,-0.259663,7.851276,3.576343,5.003662,0.658140,...,10.689589,-3.279829,-7.663726,-21.591247,-10.595358,-16.020179,1.324019,-2.008986,1.312634,1.0
1,0.528967,0.049206,-0.157453,-0.089646,-0.413954,0.777391,9.133009,3.759002,5.850874,2.083142,...,11.485905,0.587229,0.645957,-24.800812,-12.722236,-20.428559,3.286886,-1.757231,7.797418,1.0
2,-0.133676,0.182354,-0.358742,0.317244,-0.344051,0.763148,9.059090,3.811079,5.843330,-0.046902,...,-8.003641,5.558569,-1.267118,-23.902113,-13.119248,-15.446166,-2.196578,3.480885,-2.768213,1.0
3,0.131912,0.120634,0.436118,0.673803,0.053035,-0.229878,8.863699,3.966990,5.731650,-0.454538,...,-4.402870,-0.797219,-0.947221,-27.876524,-9.926772,-18.356817,-5.138227,-2.310688,6.141781,1.0
4,0.914091,0.177357,0.394977,-1.306027,-0.573817,-0.587661,8.733758,3.521825,5.351025,-1.109685,...,-0.150335,-0.842423,-0.392541,-27.829669,-10.480044,-12.421516,-1.275442,1.121175,-1.608098,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,0.546510,-0.266334,-0.469170,0.852628,-0.215050,0.805196,8.077241,4.497409,5.466102,4.069406,...,-15.229609,2.662573,3.294493,-25.867136,-22.907294,-19.654762,5.971318,-7.840835,-4.289983,0.0
988,0.299964,0.033248,-0.047572,-2.680388,-0.364900,-0.740610,9.046183,5.785707,5.694392,-1.935045,...,16.790460,10.391497,14.756330,-39.169505,-20.626036,-24.489287,-7.174058,0.435374,-4.690219,0.0
989,-0.623578,0.002080,-0.004190,1.985408,0.033497,0.936997,12.561467,6.645818,8.258492,4.941864,...,-5.998162,-14.655881,6.664632,-36.408781,-20.365514,-24.362229,-3.857486,8.711959,1.166125,0.0
990,0.667725,0.346752,0.119427,0.180092,-0.786233,0.687409,15.931164,6.279176,10.507992,0.536729,...,-18.928229,9.411250,-8.458895,-49.241454,-31.110339,-29.807670,13.366659,-12.942072,1.777476,0.0


In [4]:
x = Train_set.iloc[:,1:-1-1].values
xt =  Test_set.iloc[:,1:-1-1].values

In [5]:
y = Train_set.iloc[:,-1].values
yt= Test_set.iloc[:,-1].values

In [37]:
from sklearn.preprocessing import StandardScaler
Bhumi = StandardScaler()
x = Bhumi.fit_transform(x)


In [6]:
x_train = x
y_train = y
x_test = xt
y_test = yt

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid.fit(x_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             scoring='accuracy', verbose=1)

In [44]:


grid.best_score_

0.7596868686868687

In [25]:
from sklearn.neighbors import KNeighborsClassifier    
knn_model= KNeighborsClassifier(n_neighbors=4, metric='minkowski', p=2 )  
knn_model.fit(x_train, y_train)
y_pred= knn_model.predict(x_test)

In [30]:
from sklearn import metrics
y_pred_proba = knn_model.predict_proba(x_test)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

0.7606164383561644


In [26]:
print("Traning Accuracy : ", knn_model.score(x_train, y_train))
print("Testing Accuracy : ", knn_model.score(x_test, y_test))

Traning Accuracy :  0.8991935483870968
Testing Accuracy :  0.7330415754923414


In [27]:
y_pred = knn_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.41      0.70      0.51       184
         1.0       0.91      0.74      0.82       730

    accuracy                           0.73       914
   macro avg       0.66      0.72      0.66       914
weighted avg       0.81      0.73      0.76       914



In [28]:
from sklearn.metrics import accuracy_score
#Assign model with Decision Tree classifier
acc=[]
depth=[]
for mdepth in range(2, 10, 1):
    
    
    model1 = DecisionTreeClassifier(criterion = "gini",  #gini entropy
            random_state = 17000,max_depth=mdepth, min_samples_leaf=5)

    model1.fit(x_train, y_train)
    
    #predicting the traget variable using testing variables
    y_pred1 = model1.predict(x_test)
    #Results
    #print("Decision Tree Results")
    #print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
    acc.append(accuracy_score(y_test, y_pred1))
    depth.append(mdepth)
#printing results of loop
print(acc)
print(depth)
max_acc_index = acc.index(max(acc))  
print (max_acc_index)
print ('Optimum Depth is = ', depth[max_acc_index])
acc_max =   acc[max_acc_index]
print('Training ACC=', acc_max*100,'%')
optimum_depth = depth[max_acc_index]

#Using optimum depth value to train the model 
optimum_model = DecisionTreeClassifier(max_depth = optimum_depth) 
optimum_model.fit(x_train, y_train)

#Test the model 
y_pred1 = optimum_model.predict(x_test)
optimum_acc= accuracy_score(y_test, y_pred1)
print('Testing ACC =' ,optimum_acc*100,'%')

[0.6717724288840262, 0.74945295404814, 0.7768052516411379, 0.7177242888402626, 0.8172866520787746, 0.7866520787746171, 0.811816192560175, 0.8085339168490153]
[2, 3, 4, 5, 6, 7, 8, 9]
4
Optimum Depth is =  6
Training ACC= 81.72866520787746 %
Testing ACC = 82.16630196936543 %


In [29]:
y_pred = optimum_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.54      0.75      0.63       184
         1.0       0.93      0.84      0.88       730

    accuracy                           0.82       914
   macro avg       0.74      0.79      0.76       914
weighted avg       0.85      0.82      0.83       914



In [31]:
from sklearn import metrics
y_pred_proba = optimum_model.predict_proba(x_test)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

0.8542435973793924


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
forest= RandomForestClassifier()

n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

grid = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
grid.fit(x_train, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 8, 15, 25, 30],
                         'min_samples_leaf': [1, 2, 5, 10],
                         'min_samples_split': [2, 5, 10, 15, 100],
                         'n_estimators': [100, 300, 500, 800, 1200]},
             verbose=1)

In [9]:
grid.best_params_

{'max_depth': 30,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 100}

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(x_train,y_train)
y_pred = rf_model.predict(x_test)
print("Traning Accuracy : ", rf_model.score(x_train, y_train))
print("Testing Accuracy : ", rf_model.score(x_test, y_test))

Traning Accuracy :  1.0
Testing Accuracy :  0.8282275711159738


In [23]:
y_pred = rf_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.55      0.78      0.65       184
         1.0       0.94      0.84      0.89       730

    accuracy                           0.83       914
   macro avg       0.74      0.81      0.77       914
weighted avg       0.86      0.83      0.84       914



In [32]:
from sklearn import metrics
y_pred_proba = rf_model.predict_proba(x_test)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

0.895987194758785
